In [1]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K

class LeNet:
	@staticmethod
	def build(numChannels, imgRows, imgCols, numClasses,
		activation="relu", weightsPath=None):
		# initialize the model
		model = Sequential()
		inputShape = (imgRows, imgCols, numChannels)
		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (numChannels, imgRows, imgCols)
		# define the first set of CONV => ACTIVATION => POOL layers
		model.add(Conv2D(20, 5, padding="same",
			input_shape=inputShape))
		model.add(Activation(activation))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
		# define the second set of CONV => ACTIVATION => POOL layers
		model.add(Conv2D(50, 5, padding="same"))
		model.add(Activation(activation))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
		# define the first FC => ACTIVATION layers
		model.add(Flatten())
		model.add(Dense(500))
		model.add(Activation(activation))
		# define the second FC layer
		model.add(Dense(numClasses))
		# lastly, define the soft-max classifier
		model.add(Activation("softmax"))
		# if a weights path is supplied (inicating that the model was
		# pre-trained), then load the weights
		if weightsPath is not None:
			model.load_weights(weightsPath)
		# return the constructed network architecture
		return model

In [2]:
# import the necessary packages
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import utils
from tensorflow.keras import backend as K
import numpy as np
import cv2

In [6]:
# grab the MNIST dataset (if this is your first time running this
# script, the download may take a minute -- the 55MB MNIST dataset
# will be downloaded)

((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()
# if we are using "channels first" ordering, then reshape the
# design matrix such that the matrix is:
# num_samples x depth x rows x columns
if K.image_data_format() == "channels_first":
	trainData = trainData.reshape((trainData.shape[0], 1, 28, 28))
	testData = testData.reshape((testData.shape[0], 1, 28, 28))
# otherwise, we are using "channels last" ordering, so the design
# matrix shape should be: num_samples x rows x columns x depth
else:
	trainData = trainData.reshape((trainData.shape[0], 28, 28, 1))
	testData = testData.reshape((testData.shape[0], 28, 28, 1))
# scale data to the range of [0, 1]
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0

In [7]:
# transform the training and testing labels into vectors in the
# range [0, classes] -- this generates a vector for each label,
# where the index of the label is set to `1` and all other entries
# to `0`; in the case of MNIST, there are 10 class labels
trainLabels = utils.to_categorical(trainLabels, 10)
testLabels = utils.to_categorical(testLabels, 10)

In [8]:
# initialize the optimizer and model
opt = SGD(learning_rate=0.01)
model = LeNet.build(numChannels=1, imgRows=28, imgCols=28,numClasses=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [9]:
print("[INFO] training...")
model.fit(trainData, trainLabels, batch_size=128, epochs=20, verbose=1)

[INFO] training...
Epoch 1/20
469/469 [==============================] - 14s 5ms/step - loss: 0.9154 - accuracy: 0.7568
Epoch 2/20
469/469 [==============================] - 2s 5ms/step - loss: 0.2609 - accuracy: 0.9223
Epoch 3/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1884 - accuracy: 0.9432
Epoch 4/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1482 - accuracy: 0.9559
Epoch 5/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1236 - accuracy: 0.9629
Epoch 6/20
469/469 [==============================] - 2s 5ms/step - loss: 0.1046 - accuracy: 0.9690
Epoch 7/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0916 - accuracy: 0.9726
Epoch 8/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0822 - accuracy: 0.9751
Epoch 9/20
469/469 [==============================] - 2s 5ms/step - loss: 0.0743 - accuracy: 0.9772
Epoch 10/20
469/469 [==============================] - 2s 5ms/step - loss: 0.068

In [10]:
# show the accuracy on the testing set
print("[INFO] evaluating...")
(loss, accuracy) = model.evaluate(testData, testLabels, batch_size=128, verbose=1)
print("[INFO] accuracy: {:.2f}%".format(accuracy * 100))

[INFO] evaluating...
79/79 [==============================] - 0s 4ms/step - loss: 0.0422 - accuracy: 0.9851
[INFO] accuracy: 98.51%


In [11]:
# check to see if the model should be saved to file
print("[INFO] dumping weights to file...")
model.save_weights("weights/lenet_weights.hdf5", overwrite=True)

[INFO] dumping weights to file...


In [16]:
# randomly select a few testing digits
samples = np.random.choice(np.arange(0, len(testLabels)), size=(10,))
for i in samples:
	# classify the digit
	probs = model.predict(testData[np.newaxis, i])
	prediction = probs.argmax(axis=1)
	# extract the image from the testData if using "channels_first"
	# ordering
	if K.image_data_format() == "channels_first":
		image = (testData[i][0] * 255).astype("uint8")
	# otherwise we are using "channels_last" ordering
	else:
		image = (testData[i] * 255).astype("uint8")
	# merge the channels into one image
	image = cv2.merge([image] * 3)
	# resize the image from a 28 x 28 image to a 96 x 96 image so we
	# can better see it
	image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
	# show the image and prediction
	cv2.putText(image, str(prediction[0]), (5, 20),
				cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
	print("[INFO] Predicted: {}, Actual: {}".format(prediction[0],
		np.argmax(testLabels[i])))
	cv2.imshow("Digit", image)
	cv2.waitKey(0)

cv2.destroyAllWindows()

[INFO] Predicted: 8, Actual: 8
[INFO] Predicted: 1, Actual: 1
[INFO] Predicted: 2, Actual: 2
[INFO] Predicted: 8, Actual: 8
[INFO] Predicted: 3, Actual: 3
[INFO] Predicted: 5, Actual: 5
[INFO] Predicted: 3, Actual: 3
[INFO] Predicted: 8, Actual: 8
[INFO] Predicted: 1, Actual: 1
[INFO] Predicted: 4, Actual: 4
